In [2]:
# DB Connection
import sqlite3

# Casual
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# For Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# For Random Forest
from sklearn.ensemble import RandomForestClassifier

# NLTK
import nltk

from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import re

# Wordcloud
from wordcloud import WordCloud,STOPWORDS

# Train - Test Split
from sklearn.model_selection import train_test_split

# Testing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Dump Models
import pickle

In [2]:
#################### SZÖVEGELEMZÉS (KATEGÓRIA PREDIKTÁLÁS, ÉRZELEM OSZTÁLYOZÁS) ####################

In [3]:
# Adatbázis

connection = sqlite3.connect("podcasts.db")

In [8]:
#################### 1. KATEGÓRIA PREDIKCIÓ - TELJES ####################

In [9]:
# Adat betöltés

full_cat_pred = pd.read_sql("select reviews.content, categories.category from reviews, categories where reviews.podcast_id = categories.podcast_id", connection)
full_cat_pred_content = full_cat_pred['content'].values.tolist()
full_cat_pred_category = full_cat_pred['category'].values.tolist()

In [10]:
# Tanulás

fullcatpred = make_pipeline(TfidfVectorizer(), MultinomialNB())
fullcatpred.fit(full_cat_pred_content, full_cat_pred_category)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [11]:
# Prediktálás

def predict_category_full(s):
    pred = fullcatpred.predict([s])
    return pred

print(predict_category_full('bank system, valuta, credit card'))

['business']


In [12]:
# Mentés

pickle.dump(fullcatpred, open('fullcatpred.pkl','wb'))

In [13]:
#################### 2. KATEGÓRIA PREDIKCIÓ - RÉSZLEGES ####################

In [14]:
# Adat betöltés

par_cat_pred = pd.read_sql("select reviews.content, categories.category from reviews, categories where reviews.podcast_id = categories.podcast_id", connection)

In [15]:
# Szelektálás

categories = ['technology', 'arts-food','hinduism', 'spirituality']

def decision_maker(subject, categories):
    x = False
    for cat in categories:
        if subject == cat:
            x = True  
    return x

par_cat_pred_content = []
par_cat_pred_category = []

for ind in par_cat_pred.index:
    if (decision_maker(par_cat_pred['category'][ind], categories)):
        par_cat_pred_content.append(par_cat_pred['content'][ind])
        par_cat_pred_category.append(par_cat_pred['category'][ind])

In [16]:
# Tanulás

parcatpred = make_pipeline(TfidfVectorizer(), MultinomialNB())
parcatpred.fit(par_cat_pred_content, par_cat_pred_category)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [17]:
# Prediktálás

def predict_category_par(s):
    pred = parcatpred.predict([s])
    return pred

print(predict_category_par('hamburger, tasty, cheese, milk'))

['arts-food']


In [18]:
# Mentés

# Web alkalmazás logika

In [19]:
#################### 3. ÉRZELEM OSZTÁLYOZÁS ####################
# https://stackabuse.com/python-for-nlp-sentiment-analysis-with-scikit-learn/

In [4]:
# Adat betöltés

sentiment_data = pd.read_sql("""
    select
        content,
        case when rating < 5 then 0
            when rating > 4 then 1
        end as sentiment
    from reviews
""", connection)

sentiment_data.sentiment.value_counts()

1    996894
0    151320
Name: sentiment, dtype: int64

In [5]:
# Adat csökkentés

sentiment_data_positive = sentiment_data[sentiment_data['sentiment'] == 1].head(150000)
sentiment_data_negative = sentiment_data[sentiment_data['sentiment'] == 0].head(150000)

frames = [sentiment_data_positive, sentiment_data_negative]

sentiment = pd.concat(frames, ignore_index=True)
features = sentiment.content
labels = sentiment.sentiment

In [6]:
# Tisztítás

processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [7]:
# TF - IDF

vectorizer = TfidfVectorizer (max_features=200, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(processed_features).toarray()

In [8]:
# TF - IDF Mentés

pickle.dump(vectorizer, open('tfidf.pkl','wb'))

In [9]:
# Train - Test Split

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.05, random_state=0)

In [10]:
# Tanulás

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [11]:
# Model Mentés

pickle.dump(text_classifier, open('textclassifier.pkl','wb'))

In [12]:
# Tesztelés

predictions = text_classifier.predict(X_test)

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[5822 1625]
 [1730 5823]]
              precision    recall  f1-score   support

           0       0.77      0.78      0.78      7447
           1       0.78      0.77      0.78      7553

    accuracy                           0.78     15000
   macro avg       0.78      0.78      0.78     15000
weighted avg       0.78      0.78      0.78     15000

0.7763333333333333


In [1]:
#################### 4. SAP - Adat előkészítés és tisztítás ####################

In [4]:
ling_full = pd.read_sql("select content from reviews where length(content) < 5001", connection)

In [6]:
ling_full["id"] = ling_full.index + 1
ling_full.head()

,content,id
0,Thanks for providing these insights. Really e...,1
1,Super excited to see this podcast grow. So man...,2
2,"I'm a liberal myself, but its pretty obvious a...",3
3,I find Tedx talks very inspirational but I oft...,4
4,"I love this podcast, it is so good.",5


In [8]:
# Remove points
ling_full['content'] = ling_full['content'].str.replace('[^\w\s]','')
ling_full['content'].head()

<ipython-input-8-b009237839b1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ling_full['content'] = ling_full['content'].str.replace('[^\w\s]','')


0    Thanks for providing these insights  Really en...
1    Super excited to see this podcast grow So many...
2    Im a liberal myself but its pretty obvious and...
3    I find Tedx talks very inspirational but I oft...
4                    I love this podcast it is so good
Name: content, dtype: object

In [9]:
# Remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

ling_full['content'] = ling_full['content'].apply(lambda x: remove_emoji(x))

In [10]:
# Remove stop words
stop = stopwords.words('english')

ling_full['content'] = ling_full['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

ling_full.head(20)

,content,id
0,Thanks providing insights Really enjoy variety...,1
1,Super excited see podcast grow So many fun top...,2
2,Im liberal pretty obvious annoying theyre tryi...,3
3,I find Tedx talks inspirational I often dont t...,4
4,I love podcast good,5
5,I listened Spanish Flu pod cast After 20 minut...,6
6,Too much BS trying convince racist sexist coun...,7
7,I would love hear topics discussed without pol...,8
8,Ok I love podcast,9
9,Great podcast editors turn volume talks The in...,10


In [15]:
ling_full.to_csv('C:/Users/Zen/Desktop/Egyetem/Szakdolgozat/Text-Analysis/ling_full.csv', index = False)